In [1]:
# This script does spatial whitening, and cc in frequency domain
# Yan Yang 2022-07-10
# Haoyuan Sun 2025-06-30
from glob import glob
import numpy as np
import h5py
from time import time
import pandas as pd
import os
from func_PyCC import *
from tqdm import tqdm
from scipy.io import savemat
from fstack import Fstack

def between(lower, upper, array, mode=2):
    """
    模拟 MATLAB 的 between 函数。
    mode=2 表示包含边界。
    """
    if mode == 2:
        return np.where((array >= lower) & (array <= upper))[0]
    else:
        return np.where((array > lower) & (array < upper))[0]

def pair_to_index(i, j, n):
    """将(i, j)形式的互相关对转换为连续线性索引"""
    index = (i * (2 * n - i + 1)) // 2 + (j - i)  # 生成索引
    return index

def index_to_pair(index, n):
    """通过线性索引恢复(i, j)形式的互相关对"""
    row = 0
    while index >= (n - row):
        index -= (n - row)
        row += 1
    col = row + index
    return row, col

# 计算当前排列和前一个排列的线性索引
def reuse_index(step, n):
    index_previous = []  # 存储前一个排列的线性索引
    index_current = []   # 存储当前排列的线性索引
    index_new = [] # 需要新计算的索引

    # 计算当前排列和前一个排列的互相关对索引
    for i in range(n):
        for j in range(i, n):
            # 计算当前排列和前一个排列的线性索引
            index_current_i = pair_to_index(i + step, j + step, n)
            index_previous_i = pair_to_index(i, j, n)

            # 如果当前索引不存在于 uxt_cn2 中，说明需要新计算
            if j >= n - step:
                index_new.append(index_previous_i)
            else:
                index_current.append(index_current_i)
                index_previous.append(index_previous_i)

    # 返回索引数组
    return index_current, index_previous, index_new


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 指定处理周期
date_range = '231223_240120'
dates = [x.strftime('%Y%m%d%H') for x in pd.date_range(start='2023-12-23 15:00', end='2024-01-06 23:00', freq='1H')]

# 指定目录路径
path_preprocessed = f'/media/Data/hySun/2025_hzdf/S-preprocessed/{date_range}'
output_CC = './CC/'
if not os.path.exists(output_CC):
    os.mkdir(output_CC)

begin_channel = 0
end_channel = 2416
npair = end_channel - begin_channel + 1
#############

f1, f2 = 1, 30 # frequency band in spectral whitening
fs = 100 # sampling frequency
is_spectral_whitening = True
window_freq = 0 # 0 means aggresive spectral whitening, otherwise running mean
# max_lag = 30 # in sec, the time lag of the output CC
max_lag = 3 # in sec, the time lag of the output CC
npts_lag = int(max_lag*fs)
xcorr_seg = 60 # in sec, the length of the segment to compute CC, slightly larger than max_lag is good
npts_seg = int(xcorr_seg*fs)

device = "cuda"
gpu_ids = [0] # GPU device id
# gpu_ids = [0,1] # GPU device id
npair_chunk = 52 # depends on # of channels, sampling frequency, and xcorr_seg, needs to be adaptive
npair_chunk_sub = 26 # depends on # of channels, sampling frequency, and xcorr_seg, needs to be adaptive
overlap = 0.5  # 设置重叠度为50%
step = int(npair_chunk * (1 - overlap))  # 计算步长
step_sub = int(npair_chunk_sub * (1 - overlap))
# 更新 nchunk，以确保覆盖所有通道对
nchunk = int(np.ceil((npair - npair_chunk) / step))
nchunk_sub = int(np.ceil((npair - npair_chunk_sub) / step_sub))

# bin-stack
bin = 2
bin_x =[]
for i in range(npair_chunk):
    x = range(npair_chunk-i)
    bin_x.append(x)
x_offset = np.concatenate(bin_x, axis=0)
x_min = np.round(np.min(x_offset) / bin) * bin
x_max = np.round(np.max(x_offset) / bin) * bin
x_stack = np.arange(x_min, x_max + 2 * bin, bin)
ntrace_stack = len(x_stack)
index_list = []
for i in range(ntrace_stack):
    # 计算范围内的索引
    lower_bound = x_stack[i] - bin / 2
    upper_bound = x_stack[i] + bin / 2
    index = between(lower_bound, upper_bound, x_offset, mode=2)
    index_list.append(index)

bin_x_sub =[]
x_offset_sub_to_all = []
for i in range(npair_chunk_sub):
    x1 = range(npair_chunk_sub-i)
    bin_x_sub.append(x1)
x_offset_sub = np.concatenate(bin_x_sub, axis=0)
x_min = np.round(np.min(x_offset_sub) / bin) * bin
x_max = np.round(np.max(x_offset_sub) / bin) * bin
x_stack_sub = np.arange(x_min, x_max + 2 * bin, bin)
ntrace_stack_sub = len(x_stack_sub)
for i in range(npair_chunk_sub):
    i0 = i * np.ones((npair_chunk_sub-i))
    j0 = range(i,npair_chunk_sub)
    for j in range(len(i0)):
        x_offset_sub_to_all.append(pair_to_index(i0[j], j0[j], npair_chunk))
x_offset_sub_to_all = np.array(x_offset_sub_to_all)
index_list0 = []
index_list1 = []
index_list2 = []
for i in range(ntrace_stack_sub):
    # 计算范围内的索引
    lower_bound = x_stack_sub[i] - bin / 2
    upper_bound = x_stack_sub[i] + bin / 2
    index0 = between(lower_bound, upper_bound, x_offset_sub, mode=2)
    index0 = index0[index0 < x_offset_sub_to_all.shape[0]]
    index0 = x_offset_sub_to_all[index0]
    index1 = np.zeros((index0.shape))
    index2 = np.zeros((index0.shape))
    for j in range(len(index0)):
        pair1, pair2 = index_to_pair(index0[j], npair_chunk)
        index1[j] = pair_to_index(pair1+step_sub, pair2+step_sub, npair_chunk)
        index2[j] = pair_to_index(pair1+step_sub+step_sub, pair2+step_sub+step_sub, npair_chunk)
    index0 = index0.astype(int)
    index1 = index1.astype(int)
    index2 = index2.astype(int)
    index_list0.append(index0)
    index_list1.append(index1)
    index_list2.append(index2)
    
index_current, index_previous, index_new = reuse_index(step, npair_chunk)


/tmp/ipykernel_1112701/3376070646.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = [x.strftime('%Y%m%d%H') for x in pd.date_range(start='2023-12-23 15:00', end='2024-01-06 23:00', freq='1H')]


In [3]:
model_conf = {
    "is_spectral_whitening": is_spectral_whitening,
    "whitening_params": [fs, window_freq, f1, f2],
}
model = Torch_cross_correlation(**model_conf)
model = nn.DataParallel(model, device_ids=gpu_ids).to(device)
model.eval()

#%%
for idate in tqdm(dates):
    torch.cuda.empty_cache()
    
    output_file_path = f'{output_CC}/{date_range}/cn2_{npair_chunk}channel_{overlap}overlap/hour/'
    os.makedirs(output_file_path, exist_ok=True)
    output_file_tmp = f'{output_file_path}{idate}.mat'

    output_file_path_sub = f'{output_CC}/{date_range}/cn2_{npair_chunk_sub}channel_{overlap}overlap/hour/'
    os.makedirs(output_file_path_sub, exist_ok=True)
    output_file_tmp_sub = f'{output_file_path_sub}{idate}.mat'

    if os.path.exists(output_file_tmp):
        print(f"{idate}已存在。")
        continue

    filelist = glob(os.path.join(path_preprocessed,idate+'*h5'))
    filelist.sort()

    if len(filelist) == 0:
        print(f'{idate}: no file')
        continue

    ccall = torch.zeros((nchunk*ntrace_stack, int(max_lag * fs * 2 + 1), len(filelist)), device=device)
    ccall_sub = torch.zeros((nchunk_sub*ntrace_stack_sub, int(max_lag * fs * 2 + 1), len(filelist)), device=device)

    data_hour = []  # 用列表收集每分钟数据

    for ifile in filelist:
        try:
            with h5py.File(ifile, 'r') as fid:
                data = fid['SeisData'][:]  # 假设 data.shape 是 (n_channel, n_sample_per_minute)
                data_hour.append(data[np.newaxis, ...])  # 增加新维度，变成 (1, n_channel, n_sample)
        except Exception as e:
            print(f"跳过文件 {ifile}，读取时出错：{e}")
            continue

    data_hour = np.concatenate(data_hour, axis=0)  # 变成 (n_minute, n_channel, n_sample)

    for file_idx in range(len(filelist)):
        
        data = torch.from_numpy(data_hour[file_idx]).to(torch.float32).to(device)

        for ichunk in range(nchunk):
            start_idx = begin_channel + ichunk * step
            end_idx = start_idx + npair_chunk

            if ichunk == 0:
                data1_parts = []
                for i in range(npair_chunk):
                    row = data[start_idx + i, :].unsqueeze(0).repeat(npair_chunk - i, 1)
                    data1_parts.append(row)
                data1 = torch.cat(data1_parts, dim=0).reshape(-1, npts_seg)
                data2_parts = []
                for i in range(npair_chunk):
                    slice_data = data[start_idx + i : end_idx, :]
                    data2_parts.append(slice_data)
                data2 = torch.cat(data2_parts, dim=0).reshape(-1, npts_seg)

                cc = model(data1, data2)
                cc = cc[:,npts_seg - npts_lag - 1:npts_lag - npts_seg + 1]
                uxt_cn2 = torch.zeros((ntrace_stack, cc.shape[1]), device=device)
                for i in range(ntrace_stack):
                    uxt_cn2[i,:] = torch.sum(cc[index_list[i],:], dim=0)
                ccall[ntrace_stack * ichunk: ntrace_stack * (ichunk + 1), :, file_idx] = uxt_cn2

                uxt_cn2_sub0 = torch.zeros((ntrace_stack_sub, cc.shape[1]), device=device)
                uxt_cn2_sub1 = torch.zeros((ntrace_stack_sub, cc.shape[1]), device=device)
                uxt_cn2_sub2 = torch.zeros((ntrace_stack_sub, cc.shape[1]), device=device)
                for i in range(ntrace_stack_sub):
                    uxt_cn2_sub0[i,:] = torch.sum(cc[index_list0[i],:], dim=0)
                    uxt_cn2_sub1[i,:] = torch.sum(cc[index_list1[i],:], dim=0)
                    uxt_cn2_sub2[i,:] = torch.sum(cc[index_list2[i],:], dim=0)
                ccall_sub[ntrace_stack_sub * 2 * ichunk: ntrace_stack_sub * (2 * ichunk + 1), :, file_idx] = uxt_cn2_sub0
                ccall_sub[ntrace_stack_sub * (2 * ichunk + 1): ntrace_stack_sub * (2 * ichunk + 2), :, file_idx] = uxt_cn2_sub1
                ccall_sub[ntrace_stack_sub * (2 * ichunk + 2): ntrace_stack_sub * (2 * ichunk + 3), :, file_idx] = uxt_cn2_sub2

            else:
                cc_reuse = torch.zeros((cc.shape), device=device)
                cc_reuse[index_current,:] = cc[index_previous,:]

                data1_parts = []
                for i in range(npair_chunk):
                    row = data[start_idx + i, :].unsqueeze(0).repeat(npair_chunk - i, 1)
                    data1_parts.append(row)
                data1 = torch.cat(data1_parts, dim=0).reshape(-1, npts_seg)
                data1 = data1[index_new,:]
                data2_parts = []
                for i in range(npair_chunk):
                    slice_data = data[start_idx + i : end_idx, :]
                    data2_parts.append(slice_data)
                data2 = torch.cat(data2_parts, dim=0).reshape(-1, npts_seg)
                data2 = data2[index_new,:]

                cc = model(data1, data2)
                cc = cc[:,npts_seg - npts_lag - 1:npts_lag - npts_seg + 1]
                cc_reuse[index_new,:] = cc
                cc = cc_reuse
                uxt_cn2 = torch.zeros((ntrace_stack, cc.shape[1]), device=device)
                for i in range(ntrace_stack):
                    uxt_cn2[i,:] = torch.sum(cc[index_list[i],:], dim=0)
                ccall[ntrace_stack * ichunk: ntrace_stack * (ichunk + 1), :, file_idx] = uxt_cn2
                
                uxt_cn2_sub1 = torch.zeros((ntrace_stack_sub, cc.shape[1]), device=device)
                uxt_cn2_sub2 = torch.zeros((ntrace_stack_sub, cc.shape[1]), device=device)
                for i in range(ntrace_stack_sub):
                    uxt_cn2_sub1[i,:] = torch.sum(cc[index_list1[i],:], dim=0)
                    uxt_cn2_sub2[i,:] = torch.sum(cc[index_list2[i],:], dim=0)
                ccall_sub[ntrace_stack_sub * (2 * ichunk + 1): ntrace_stack_sub * (2 * ichunk + 2), :, file_idx] = uxt_cn2_sub1
                ccall_sub[ntrace_stack_sub * (2 * ichunk + 2): ntrace_stack_sub * (2 * ichunk + 3), :, file_idx] = uxt_cn2_sub2

    ccall = Fstack(ccall, stack_flag=1, nu=1, dim=2, device=device).cpu().numpy()
    ccall_sub = Fstack(ccall_sub, stack_flag=1, nu=1, dim=2, device=device).cpu().numpy()

    savemat(output_file_tmp, {'data': ccall})
    savemat(output_file_tmp_sub, {'data': ccall_sub})


  0%|          | 0/345 [00:00<?, ?it/s]

2023122315已存在。
2023122316已存在。
2023122317已存在。
2023122318已存在。
2023122319已存在。
2023122320已存在。
2023122321已存在。
2023122322已存在。
2023122323已存在。
2023122400已存在。
2023122401已存在。
2023122402已存在。
2023122403已存在。
2023122404已存在。
2023122405已存在。
